In [58]:
# Load libraries and global parameters
packages =  c("lubridate","Hmisc", "ggplot2","directlabels", "dplyr", "tidyr", "data.table", 'corrplot', 'gridExtra', 'forecast', 'tseries', 'TSA', 'tibble', 'TTR', 'xts', 'dygraphs', 'assertthat')

my.install <- function(pkg, ...){
  if (!(pkg %in% installed.packages()[,1])) {
    install.packages(pkg)
  }
  return (library(pkg, ...))
}

purrr::walk(packages, my.install, character.only = TRUE, warn.conflicts = FALSE)

In [57]:
crime.data = read.csv('crime_report.csv')
str(crime.data)

'data.frame':	2829 obs. of  15 variables:
 $ report_year        : int  1975 1975 1975 1975 1975 1975 1975 1975 1975 1975 ...
 $ agency_code        : Factor w/ 69 levels "","AZ00717","AZ00723",..: 42 59 22 14 61 30 33 29 44 39 ...
 $ agency_jurisdiction: Factor w/ 69 levels "Albuquerque, NM",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ population         : int  286238 112478 490584 116656 300400 642154 864100 616120 422276 262103 ...
 $ violent_crimes     : int  2383 278 8033 611 1215 1259 16086 11386 3350 1937 ...
 $ homicides          : int  30 5 185 7 33 25 259 119 63 68 ...
 $ rapes              : int  181 28 443 44 190 137 463 453 192 71 ...
 $ assaults           : int  1353 132 3518 389 463 347 6309 3036 755 976 ...
 $ robberies          : int  819 113 3887 171 529 750 9055 7778 2340 822 ...
 $ months_reported    : int  12 12 12 12 12 12 12 12 12 12 ...
 $ crimes_percapita   : num  833 247 1637 524 404 ...
 $ homicides_percapita: num  10.48 4.45 37.71 6 10.99 ...
 $ rapes_percapita    : num  6

In [3]:
#Overview of data
summary(crime.data)
str(crime.data)

  report_year    agency_code             agency_jurisdiction   population     
 Min.   :1975          :  41   Albuquerque, NM     :  41     Min.   : 100763  
 1st Qu.:1985   AZ00717:  41   Arlington, TX       :  41     1st Qu.: 377931  
 Median :1995   AZ00723:  41   Atlanta, GA         :  41     Median : 536614  
 Mean   :1995   AZ01003:  41   Aurora, CO          :  41     Mean   : 795698  
 3rd Qu.:2005   CA00109:  41   Austin, TX          :  41     3rd Qu.: 816856  
 Max.   :2015   CA01005:  41   Baltimore County, MD:  41     Max.   :8550861  
                (Other):2583   (Other)             :2583     NA's   :69       
 violent_crimes      homicides           rapes           assaults    
 Min.   :    154   Min.   :    1.0   Min.   :  15.0   Min.   :   15  
 1st Qu.:   3015   1st Qu.:   32.0   1st Qu.: 176.2   1st Qu.: 1467  
 Median :   5136   Median :   64.0   Median : 291.0   Median : 2597  
 Mean   :  29633   Mean   :  398.4   Mean   : 416.3   Mean   : 4405  
 3rd Qu.:   9058  

'data.frame':	2829 obs. of  15 variables:
 $ report_year        : int  1975 1975 1975 1975 1975 1975 1975 1975 1975 1975 ...
 $ agency_code        : Factor w/ 69 levels "","AZ00717","AZ00723",..: 42 59 22 14 61 30 33 29 44 39 ...
 $ agency_jurisdiction: Factor w/ 69 levels "Albuquerque, NM",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ population         : int  286238 112478 490584 116656 300400 642154 864100 616120 422276 262103 ...
 $ violent_crimes     : int  2383 278 8033 611 1215 1259 16086 11386 3350 1937 ...
 $ homicides          : int  30 5 185 7 33 25 259 119 63 68 ...
 $ rapes              : int  181 28 443 44 190 137 463 453 192 71 ...
 $ assaults           : int  1353 132 3518 389 463 347 6309 3036 755 976 ...
 $ robberies          : int  819 113 3887 171 529 750 9055 7778 2340 822 ...
 $ months_reported    : int  12 12 12 12 12 12 12 12 12 12 ...
 $ crimes_percapita   : num  833 247 1637 524 404 ...
 $ homicides_percapita: num  10.48 4.45 37.71 6 10.99 ...
 $ rapes_percapita    : num  6

In [4]:
#Data Cleaning
crime.data[!complete.cases(crime.data),]
nrow(crime.data[!complete.cases(crime.data),])

,report_year,agency_code,agency_jurisdiction,population,violent_crimes,homicides,rapes,assaults,robberies,months_reported,crimes_percapita,homicides_percapita,rapes_percapita,assaults_percapita,robberies_percapita
,<int>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
31,1975,KY05680,"Louisville, KY",NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
69,1975,,United States,NA,1039710,20510,NA,NA,NA,NA,487.8,9.60,NA,NA,NA
100,1976,KY05680,"Louisville, KY",NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
138,1976,,United States,NA,1004210,18780,NA,NA,NA,NA,467.8,8.70,NA,NA,NA
169,1977,KY05680,"Louisville, KY",NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
207,1977,,United States,NA,1029580,19120,NA,NA,NA,NA,475.9,8.80,NA,NA,NA
238,1978,KY05680,"Louisville, KY",NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
276,1978,,United States,NA,1085550,19560,NA,NA,NA,NA,497.8,9.00,NA,NA,NA
307,1979,KY05680,"Louisville, KY",NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


[1] 141

The total records having missing values is 141. Upon suspection, the column months_reported has the most missing values. Since it is not significant in the research, it is removed. After removal, incomplete cases reduced to 76. These 76 records were then removed rather than imputed since the data used is very sensitive and is unethical to assume a crime rate for a particular area.

In [5]:
# remove the column month_reported
crime.data = select(crime.data, -c(months_reported))
nrow(crime.data[!complete.cases(crime.data),])

[1] 76

In [6]:
# remove records with missing values
crime.data = na.omit(crime.data)
summary(crime.data)

  report_year    agency_code        agency_jurisdiction   population     
 Min.   :1975   AZ00717:  41   Albuquerque, NM:  41     Min.   : 100763  
 1st Qu.:1985   AZ00723:  41   Arlington, TX  :  41     1st Qu.: 378014  
 Median :1995   AZ01003:  41   Atlanta, GA    :  41     Median : 537081  
 Mean   :1995   CA00109:  41   Aurora, CO     :  41     Mean   : 796458  
 3rd Qu.:2005   CA01005:  41   Austin, TX     :  41     3rd Qu.: 817083  
 Max.   :2015   CA01900:  41   Baltimore, MD  :  41     Max.   :8550861  
                (Other):2507   (Other)        :2507                      
 violent_crimes     homicides          rapes           assaults    
 Min.   :   154   Min.   :   1.0   Min.   :  15.0   Min.   :   15  
 1st Qu.:  2965   1st Qu.:  32.0   1st Qu.: 176.0   1st Qu.: 1467  
 Median :  5066   Median :  63.0   Median : 291.0   Median : 2597  
 Mean   :  8946   Mean   : 124.9   Mean   : 416.2   Mean   : 4405  
 3rd Qu.:  8804   3rd Qu.: 127.0   3rd Qu.: 465.0   3rd Qu.: 4556  


In [7]:
#Extracting crime columns from the data
crime.data.crime = crime.data[,c(5,6,7,8,9)]
summary(crime.data.crime)

 violent_crimes     homicides          rapes           assaults    
 Min.   :   154   Min.   :   1.0   Min.   :  15.0   Min.   :   15  
 1st Qu.:  2965   1st Qu.:  32.0   1st Qu.: 176.0   1st Qu.: 1467  
 Median :  5066   Median :  63.0   Median : 291.0   Median : 2597  
 Mean   :  8946   Mean   : 124.9   Mean   : 416.2   Mean   : 4405  
 3rd Qu.:  8804   3rd Qu.: 127.0   3rd Qu.: 465.0   3rd Qu.: 4556  
 Max.   :174542   Max.   :2245.0   Max.   :3899.0   Max.   :71030  
   robberies     
 Min.   :    83  
 1st Qu.:  1032  
 Median :  1939  
 Mean   :  4000  
 3rd Qu.:  3609  
 Max.   :107475  

In [52]:
#crime.data.crime = aggregate(crime.data[,c(5,6,7,8,9)], by=list(crime.data$agency_jurisdiction), FUN=sum)
#total = rowSums(crime.data.crime[,c(2,3,4,5,6)])
#crime.data.crime = cbind(crime.data.crime, total)
#crime.data.crime
total = rowSums(crime.data[,c(5,6,7,8,9)])
crime.data = cbind(crime.data, total)

In [ ]:
library(directlabels)
ggplot( data = crime.data, aes(x=report_year, y=total, group=agency_jurisdiction, color=agency_jurisdiction)) +
    geom_line() + 
    geom_dl(aes(label=agency_jurisdiction), method="last.points") 


#graph shows in R

In [59]:
#Add date column for ggplot time series
library(lubridate)
report_date <- as.Date(as.character(crime.data$report_year), format = "%Y")
cbind(crime.data, report_date)

report_year,agency_code,agency_jurisdiction,population,violent_crimes,homicides,rapes,assaults,robberies,months_reported,crimes_percapita,homicides_percapita,rapes_percapita,assaults_percapita,robberies_percapita,report_date
<int>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<date>
1975,NM00101,"Albuquerque, NM",286238,2383,30,181,1353,819,12,832.52,10.48,63.23,472.68,286.13,1975-12-05
1975,TX22001,"Arlington, TX",112478,278,5,28,132,113,12,247.16,4.45,24.89,117.36,100.46,1975-12-05
1975,GAAPD00,"Atlanta, GA",490584,8033,185,443,3518,3887,12,1637.44,37.71,90.30,717.10,792.32,1975-12-05
1975,CO00101,"Aurora, CO",116656,611,7,44,389,171,12,523.76,6.00,37.72,333.46,146.58,1975-12-05
1975,TX22701,"Austin, TX",300400,1215,33,190,463,529,12,404.46,10.99,63.25,154.13,176.10,1975-12-05
1975,MD00301,"Baltimore County, MD",642154,1259,25,137,347,750,12,196.06,3.89,21.33,54.04,116.79,1975-12-05
1975,MDBPD00,"Baltimore, MD",864100,16086,259,463,6309,9055,12,1861.59,29.97,53.58,730.12,1047.91,1975-12-05
1975,MA01301,"Boston, MA",616120,11386,119,453,3036,7778,12,1848.02,19.31,73.52,492.76,1262.42,1975-12-05
1975,NY01401,"Buffalo, NY",422276,3350,63,192,755,2340,12,793.32,14.92,45.47,178.79,554.14,1975-12-05
